[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/boruizhang/newa)


https://github.com/boruizhang/newa.git

In [7]:
#uncommon for clone repo
#gh repo clone boruizhang/newa

In [8]:
in_s = 'दुहां वइगु ध्यबाया परिमाण अप्वइगु खँ नं अर्थविद्तय्सं धाःगु दु । डलरया भाः थहां वन कि विदेशय् ज्या यानाच्वंपिं नेपाःमितय्सं पेश्की हे तलब कयाः तक नं नेपालय् ध्यबा छ्वा हइगु जूगुलिं आः छुं ई रेमिटेन्स अप्वयेफुगु अनुमान नेपाल राष्ट्र बैंकं याःगु दु ।'
out = in_s.split('। ')[:-1]
print('\n'.join(out))

दुहां वइगु ध्यबाया परिमाण अप्वइगु खँ नं अर्थविद्तय्सं धाःगु दु 


# 1. clean out gibberish

In [ ]:
# this step shinks 5441 to 5017 lines
text_file = '/Users/bri/dev/NewarOscar/new_dedup.txt'

bad_words = ['Ü','ú', 'ö', 'ó','ő','í','É']

with open(text_file) as oldfile, open('newfile.txt', 'w') as newfile:
    for line in oldfile:
        if not any(bad_word in line for bad_word in bad_words):
            newfile.write(line)

# 2. put each sentence in one line

In [ ]:
TextFile = '/Users/bri/dev/NewarOscar/newfile.txt'

with open(TextFile, 'r') as oldfile, open('lined_up.txt','w') as lined_up:
    for line in oldfile:
        out = line.split('।')
        if out != "":
            lined_up.write('\n'.join(out))

# 3. regex remove empty lines (with Atom) 

In [ ]:
# result in 16694

# 4. remove too short lines 

In [ ]:
# result 16603 lines
TextFile = '/Users/bri/dev/NewarOscar/lined_up.txt'

with open(TextFile, 'r') as oldfile, open('good_length.txt','w') as good_length:
    for line in oldfile:
        if len(line)>3:
            good_length.write(line)

In [ ]:
# delete symbols [...] 16599

## get all ki-sentences

In [ ]:
# ki-sentences 2660 raw
TextFile = '/Users/bri/dev/NewarOscar/good_length.txt'

with open(TextFile, 'r') as oldfile, open('ki.txt','w') as ki_sents:
    for line in oldfile:
        if "कि" in line:
            ki_sents.write(line)
          #  print (len(line))

In [ ]:
# ki-sentences without ,
TextFile = '/Users/bri/dev/NewarOscar/ki.txt'

punc = [',','य्‌']

with open(TextFile, 'r') as oldfile, open('ki_no_punc.txt','w') as ki_no_punc:
    for line in oldfile:
        if not any(elem in line for elem in punc):
            ki_no_punc.write(line)
          #  print (len(line))

## get all dhaka-sentences

In [ ]:
# dhaka-sentences 682 raw
TextFile = '/Users/bri/dev/NewarOscar/good_length.txt'

with open(TextFile, 'r') as oldfile, open('dhaka.txt','w') as dhaka_sents:
    for line in oldfile:
        if "धका" in line:
            dhaka_sents.write(line)
          #  print (len(line))

# 5. Whitespace Tokenizing

In [2]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()
	
# tokenize cleaned raw corpus file
with open("good_length.txt") as f, open('good_length_tok.txt','w') as tok:
    content = f.read() # Read the whole file
    lines = content.split('\n') # a list of all sentences
    for line in lines:
        tokens = tk.tokenize(line)
        tok.write(str(tokens))


# 6. formating training data from native speakers' annotation file

In [46]:
# linux command lines:
# regex space out [] from adjacent words
# regex delete sentence number indexes
# command line to delete original sentence (for example:)
# awk 'NR % 4 != 1' 0_100_annotated-touch.txt > 0_100_dhaka_true_withEn.txt
# awk 'NR % 3 != 0' 0_100_dhaka_true_withEn.txt > 0_100_dhaka_true.txt

In [ ]:
#human annotated format example
sent_1 = '[ फ्रान्सया मेपिं विद्वानतय्सं वय्कःया थ्व सफू व उकिया विषयवस्तुयात न्हूगु विषय धकाः ] ययेकादीगु खँ नं वय्कलं कनादीगु दु'
sent_2 = ['कनादीगु',"UNK"]
    
#labels:

#BEC
#IEC
#OEC
#OEC-V
#IEC-V

## Getting NER output in Conll2003 styled

In [9]:
## see output in file "total_true_emb" in repo

import itertools
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()
with open('dhaka_true.txt') as f:
#with open('ki_true.txt') as f:
    for line1,line2 in itertools.zip_longest(*[f]*2):
        tokens = iter(tk.tokenize(line1))
        current_tag = "OEC"
        for token in tokens:
            if token in line2:    
                # Good: even if the verb is complex (aka spaced out),suffix can still apply to the entire phrase
                # Bad: false positive -V may occur, if one word that has multiple meanings occurring more than one time in the same sentence
                suffix = "-V"
            else:
                suffix = ""
            if token == "[":
                print(next(tokens), "\t"+"BEC"+suffix)
                current_tag = "IEC"
            elif token == "]":
                current_tag = "OEC"
                continue
            else:
                print(token, current_tag+suffix, sep="\t")
        print("")
        

४७ 	BEC-V
सालय्	IEC
संविधानय्	IEC
धर्मनिरपेक्षता	IEC
तयेमाः	IEC-V
धकाः	IEC
याःगु	OEC
आन्दोलन	OEC
झीसं	OEC
ल्वःमंकागु	OEC-V
मदुनि	OEC

अझ 	BEC-V
अन्तरिम	IEC
संविधानय्	IEC
धर्म	IEC
निरपेक्ष	IEC
धकाः	IEC
च्वयेधुंकूगु	OEC-V
हे	OEC
आः	OEC
वयाः	OEC
सनातन	OEC
धर्मयात	OEC
जक	OEC
विशेष	OEC
संरक्षण	OEC
जुइकथंया	OEC
व्याख्या	OEC
जूवंगु	OEC
दु	OEC

थ्व	OEC
खँ	OEC
विश्वलय्	OEC
तकं	OEC
बय्बय्	OEC
जुइगु	OEC
जुल,	OEC
नेपाः 	BEC-V
सरकार	IEC
बुद्धधर्म	IEC
व	IEC
बौद्ध	IEC
जनताया	IEC
मन	IEC
तयेत	IEC
हे	IEC
जूसां	IEC
बुद्धया	IEC
दर्शनया	IEC
गाम्भीर्ययात	IEC
नालाकयाः	IEC
थज्याःगु	IEC
ज्या	IEC-V
याःगु	IEC-V
धकाः	IEC
तायेकीगु	OEC-V
जुल	OEC

सात 	BEC-V
करोड	IEC
खर्च	IEC-V
जूगु	IEC
धकाः	IEC
बय्बय्	OEC
यानातःगु	OEC-V
उगु	OEC
सम्मेलनया	OEC
उपलब्धि	OEC
छु	OEC
जुल	OEC
धकाः	OEC
नेपाःया	OEC
गुगुं	OEC
बौद्ध	OEC
संघसंस्थां	OEC
न्यंगु,	OEC
अले	OEC
सरकारं	OEC
लिसः	OEC
बिउगु	OEC
न्यनेगु	OEC
सौभाग्य	OEC
चूलाःगु	OEC
मदुनि	OEC

सरकारया 	BEC-V
निंतिं	IEC
अज्याःगु	IEC
अवसर	IEC
धइगु	IEC
बुद्धया	IEC
नामय्	IEC
थुलि	IEC
ध्यबा	IEC


वैज्ञानिक	IEC
आधार	IEC
व	IEC-V
दृष्टिकोण	IEC
नं	IEC
ब्यागः	IEC
मरु	IEC
धकाः	IEC
बांलाकु	OEC
हे	OEC
धाये	OEC-V
छिं	OEC-V

थन 	BEC-V
स्वनिगः	IEC
व	IEC
हिमाली	IEC
भेग	IEC
अले	IEC
मेगु	IEC
छगू	IEC
निगू	IEC
थासय्	IEC
बाहेक	IEC
मेथाय्	IEC
बुद्ध	IEC
धइम्ह	IEC
गुम्ह	IEC
झंगःया	IEC
नां	IEC
धकाः	IEC
तकं	OEC
मसिउ	OEC-V

थन 	BEC-V
नं	IEC
ला	IEC
चिकुला	IEC
धइगु	IEC
दु,	IEC
तराइया	IEC
चिकुला	IEC
गुलि	IEC
चिकुइ	IEC
धकाः	IEC
नं	OEC
मसिउगु	OEC-V
मखु	OEC

चित्लाङ्गया 	BEC-V
थ्यंमथ्यं	IEC
९०	IEC
छेँया	IEC
बलामी	IEC
जातिं	IEC
संरक्षण	IEC
यानावयाच्वंगु	IEC-V
थुगु	IEC
रामसीताया	IEC
जात्रायात	IEC
महाचुनी	IEC
जात्रा	IEC
धकाः	IEC
धायेगु	OEC
यानावयाच्वंगु	OEC-V
दु	OEC-V

थनया 	BEC-V
आदिवासी	IEC
रैथाने	IEC
नेवाः	IEC
समुदाययात	IEC
सरकारं	IEC
विस्थापित	IEC-V
याइ	IEC
मखु	IEC
धकाः	IEC
धाःसां	OEC
थुकिं	OEC
मचाय्क	OEC
विस्थापित	OEC-V
याना	OEC
यंकीगु	OEC
व	OEC-V
चक्रपथ	OEC
दयेकाः	OEC
न्ह्याइगु	OEC
मोटर	OEC
गाडीत	OEC
पाखें	OEC
काइगु	OEC
करया	OEC
रोयल्टी	OEC
जग्गाथुवाःतय्त	OEC
दै	OEC
कि	OEC
मदै	OEC
धैगु	OEC
विषयसं	OEC
सह

मखु	OEC

अझ 	BEC-V
वृहद	IEC
कथं	IEC
धायेगु	IEC
खःसा	IEC
थ्व	IEC
विज्ञान	IEC
ब्रह्माण्ड	IEC
गथे	IEC
सनाच्वंगु	IEC-V
दु	IEC
धका	IEC
स्वयेत	OEC-V
याइगु	OEC
प्रकृतिया	OEC
साधारण	OEC
एनालाइसिस	OEC
ख	OEC

2अले 	BEC-V
इमिसं	IEC
मनूतय्‌के	IEC
थथे	IEC
धकाः	IEC
न्‍यन	OEC-V
--	OEC
“यहूदीतय्‌	OEC
जुजु	OEC
जुयाः	OEC
बूम्‍ह	OEC
गन	OEC
दु?	OEC
वय्‌कः	OEC
बूगु	OEC
खँ	OEC
सीकेगु	OEC
छगः	OEC
नगु	OEC
पुर्बय्‌	OEC
खनागुलिं	OEC
वय्‌कःयात	OEC
दर्शन	OEC
यायेत	OEC
जिपिं	OEC
थन	OEC
वयागु	OEC
खः	OEC

4उकिं	OEC
हेरोद	OEC
जुजुं	OEC
तःधंपिं	OEC
पुजाहारीतय्‌त	OEC
व	OEC-V
शास्‍त्रीतय्‌त	OEC
सःताः	OEC-V
थथे	OEC
न्‍यन	OEC
--	OEC
“मुक्ति	OEC
बीम्‍ह	OEC
ख्रीष्‍ट	OEC
गन	OEC
बुइगु?”	OEC
5अले	OEC
इमिसं	OEC
थथे	OEC
धकाः 	BEC-V
लिसः	IEC
बिल	IEC
--	IEC
“यहूदियायागु	IEC
बेथलेहेमय्‌	IEC
दु,	IEC
छाय्‌धाःसा	IEC
थ्‍व	IEC
खँ	IEC
छम्‍ह	IEC
अगमवक्तां	IEC
नं	IEC
थथे	IEC
च्‍वया	IEC-V
थकूगु	IEC
दु	IEC
--	IEC
6‘हे	IEC
यहूदियाया	IEC
बेथलेहेम,	IEC
छ	IEC
यहूदियाया	IEC
जुजुपिनिगु	IEC
न्‍ह्यःने	IEC
छुं	IEC
कथं	IEC
नं	IEC
चिधं	IEC
मजू,	IEC
छाय

खुसिइ	IEC
मच्वंगु	IEC-V
व	IEC-V
लिपा	IEC
एनियसया	IEC
वंशजत	IEC
अल्बान	IEC
(Alban)	IEC
जुजु	IEC
जूगु	IEC
व	IEC-V
थ्व	IEC
हे	IEC
अल्बान	IEC
वंशजं	IEC
रोमुलस	IEC
बुयावःगु	IEC
धका	IEC
च्वयातःगु	OEC-V
दु	OEC-V

छुं 	BEC-V
छगु	IEC
रेजिस्तरय्	IEC
थ्व	IEC
रेजिस्त्यान्स	IEC
भोल्तेज	IEC
स्वया	IEC-V
स्वतन्त्र	IEC
व	IEC-V
मपाइगु	IEC
(constant)	IEC
धका	IEC
नालेगु	OEC-V
या	OEC-V

थ्व 	BEC-V
विजय	IEC
बेल्जियमं	IEC
विश्वकप	IEC
त्याकेफइ	IEC-V
ला	IEC
धकाः	IEC
न्ह्यसः	OEC
तयाच्वनीपिं	OEC
आलोचकतय्	OEC
निंतिं	OEC
नं	OEC
बांलाःगु	OEC
लिसः	OEC
जूगु	OEC-V
दु	OEC-V

दसुया 	BEC-V
निंतिं	IEC
नाउरु	IEC
देय्‌या	IEC
जानकारी	IEC
कायेत	IEC
छिं	IEC
च्वेयागु	IEC-V
सन्दूकय्	IEC
नाउरु	IEC
धका	IEC
तिया	OEC
मालादिसँ	OEC-V

वय्कलं 	BEC-V
धयादी,	IEC
‘जिमि	IEC
म्ह्याय्या	IEC
नामं	IEC
मेपिं	IEC
नं	IEC
बौद्धिक	IEC
अपांग	IEC
जूपिं	IEC-V
मिसा	IEC
मस्तय्गु	IEC
भविष्य	IEC
बांलायेमा	IEC
धकाः	IEC
थ्व	OEC
सुनगाभा	OEC
होम्स	OEC
शुरु	OEC-V
यानागु	OEC-V
खः	OEC

थुकिया 	BEC-V
ग्वहालि	IEC
कया	IEC
ब्वमिं	IEC
बाखनय्	IEC
लिपा	IEC
छु	IEC
जुइ	I

In [ ]:
# distilbert-base-multilingual-cased has Newar

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)
pip install inltk
conda install -c conda-forge spacy
from inltk.inltk import identify_language
identify_language("中文")
from nltk.parse.corenlp import CoreNLPDependencyParser